In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
Train_data = pd.read_csv('/content/fraudTrain.csv')
Test_data  = pd.read_csv('/content/fraudTest.csv')

In [3]:
Train_data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495.0,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1.325376e+09,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149.0,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1.325376e+09,49.159047,-118.186462,0.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154.0,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1.325376e+09,43.150704,-112.154481,0.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939.0,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1.325376e+09,47.034331,-112.561071,0.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99.0,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1.325376e+09,38.674999,-78.632459,0.0


In [4]:
X_Train = Train_data.drop(columns='is_fraud', axis=1)
Y_Train = Train_data['is_fraud']

In [5]:
X_Test= Train_data.drop(columns='is_fraud', axis=1)
Y_Test = Train_data['is_fraud']

In [21]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

# Define categorical features
one_hot_features = ['gender']  # Suitable for OneHotEncoder
label_encoding_features = ['job']  # Example of Label Encoding
frequency_encoding_features = ['merchant', 'street', 'first', 'last', 'trans_num', 'category', 'state' , 'city' , 'dob' , 'trans_date_trans_time' , 'gender']  # Example of Frequency Encoding

# Frequency Encoding
for col in frequency_encoding_features:
    freq = X_Train[col].value_counts()  # Frequency count
    X_Train[col] = X_Train[col].map(freq)

# Label Encoding
label_encoders = {}
for col in label_encoding_features:
    le = LabelEncoder()
    X_Train[col] = le.fit_transform(X_Train[col])
    label_encoders[col] = le  # Store encoders for later use on test data

# One-Hot Encoding
transformer = ColumnTransformer(
    [('one_hot', OneHotEncoder(), one_hot_features)],
    remainder='passthrough'
)

# Transform One-Hot Features
transformed_X_Train = transformer.fit_transform(X_Train)


In [22]:
print(Train_data.isnull().sum())


Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   1
street                   1
city                     1
state                    1
zip                      1
lat                      1
long                     1
city_pop                 1
job                      1
dob                      1
trans_num                1
unix_time                1
merch_lat                1
merch_long               1
is_fraud                 1
dtype: int64


In [23]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer

# Handling Y_Train (Target)
# Impute missing values in the target variable
y_imputer = SimpleImputer(strategy='most_frequent')  # Replace NaN in Y_Train with mode
Y_Train = pd.DataFrame(y_imputer.fit_transform(Y_Train.values.reshape(-1, 1)), columns=['target']).squeeze()

# Handling X_Train (Features)
# Identify numerical and categorical features
numerical_features = X_Train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_Train.select_dtypes(include=['object']).columns

# Create imputers for numerical and categorical data
numerical_imputer = SimpleImputer(strategy='mean')  # Replace NaN with mean for numerical data
categorical_imputer = SimpleImputer(strategy='most_frequent')  # Replace NaN with mode for categorical data

# Preprocessor for X_Train
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_imputer, numerical_features),
        ('cat', categorical_imputer, categorical_features)
    ]
)

# Create a pipeline with preprocessing and Logistic Regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Fit the pipeline to the data
#pipeline.fit(X_Train, Y_Train)

# Use the pipeline for predictions
# predictions = pipeline.predict(Test_data)


In [24]:
X_Train = X_Train.dropna()
Y_Train = Y_Train.loc[X_Train.index]  # Ensure labels match the new X_Train


In [25]:
X_Train = X_Train.dropna(axis=1)


In [26]:
from sklearn.ensemble import HistGradientBoostingClassifier

# Use a model that handles NaN values
model = HistGradientBoostingClassifier()
model.fit(X_Train, Y_Train)

# predictions = model.predict(X_Test)


HistGradientBoostingClassifier()

In [28]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=5)
X_Train = pd.DataFrame(knn_imputer.fit_transform(X_Train), columns=X_Train.columns)


In [29]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iterative_imputer = IterativeImputer()
X_Train = pd.DataFrame(iterative_imputer.fit_transform(X_Train), columns=X_Train.columns)


In [30]:
print(X_Train.isnull().sum())


Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
dtype: int64


In [37]:
Test_data.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1.371817e+09,33.986391,-81.200714,0.0
1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1.371817e+09,39.450498,-109.960431,0.0
2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1.371817e+09,40.495810,-74.196111,0.0
3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1.371817e+09,28.812398,-80.883061,0.0
4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1.371817e+09,44.959148,-85.884734,0.0


In [38]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
import pandas as pd

# Define categorical features
one_hot_features = ['gender']  # Suitable for OneHotEncoder
label_encoding_features = ['job']  # Example of Label Encoding
frequency_encoding_features = ['merchant', 'street', 'first', 'last', 'trans_num', 'category', 'state' , 'city' , 'dob' , 'trans_date_trans_time' , 'gender']  # Example of Frequency Encoding

# Frequency Encoding
for col in frequency_encoding_features:
    freq = X_Test[col].value_counts()  # Frequency count
    X_Test[col] = X_Test[col].map(freq)

# Label Encoding
label_encoders = {}
for col in label_encoding_features:
    le = LabelEncoder()
    X_Test[col] = le.fit_transform(X_Test[col])
    label_encoders[col] = le  # Store encoders for later use on test data

# One-Hot Encoding
transformer = ColumnTransformer(
    [('one_hot', OneHotEncoder(), one_hot_features)],
    remainder='passthrough'
)

# Transform One-Hot Features
transformed_X_Test = transformer.fit_transform(X_Test)


In [39]:
print(Test_data.isnull().sum())

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      1
trans_num                1
unix_time                1
merch_lat                1
merch_long               1
is_fraud                 1
dtype: int64


In [40]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer

# Handling Y_Train (Target)
# Impute missing values in the target variable
y_imputer = SimpleImputer(strategy='most_frequent')  # Replace NaN in Y_Train with mode
Y_Test = pd.DataFrame(y_imputer.fit_transform(Y_Test.values.reshape(-1, 1)), columns=['target']).squeeze()

# Handling X_Train (Features)
# Identify numerical and categorical features
numerical_features = X_Test.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_Test.select_dtypes(include=['object']).columns

# Create imputers for numerical and categorical data
numerical_imputer = SimpleImputer(strategy='mean')  # Replace NaN with mean for numerical data
categorical_imputer = SimpleImputer(strategy='most_frequent')  # Replace NaN with mode for categorical data

# Preprocessor for X_Train
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_imputer, numerical_features),
        ('cat', categorical_imputer, categorical_features)
    ]
)

# Create a pipeline with preprocessing and Logistic Regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Fit the pipeline to the data
#pipeline.fit(X_Train, Y_Train)

# Use the pipeline for predictions
# predictions = pipeline.predict(Test_data)


In [41]:
X_Test = X_Test.dropna()
Y_Test = Y_Test.loc[X_Test.index]  # Ensure labels match the new X_Train

In [43]:
X_Test = X_Test.dropna(axis=1)


In [44]:
from sklearn.ensemble import HistGradientBoostingClassifier

# Use a model that handles NaN values
model = HistGradientBoostingClassifier()
model.fit(X_Test, Y_Test)

# predictions = model.predict(X_Test)

HistGradientBoostingClassifier()

In [45]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=5)
X_Test = pd.DataFrame(knn_imputer.fit_transform(X_Test), columns=X_Test.columns)


In [46]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iterative_imputer = IterativeImputer()
X_Test = pd.DataFrame(iterative_imputer.fit_transform(X_Test), columns=X_Test.columns)


In [47]:
print(X_Test.isnull().sum())

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
dtype: int64


In [48]:
model = LogisticRegression()

In [49]:
model.fit(X_Train, Y_Train)

LogisticRegression()

In [50]:
#accuracy on training data
X_Train_prediction = model.predict(X_Train)
training_data_accuracy = accuracy_score(X_Train_prediction, Y_Train)

In [51]:
print('Accuracy on training data : ', training_data_accuracy)

Accuracy on training data :  0.9942411057077041


In [52]:
#accuracy on test data
X_Test_prediction = model.predict(X_Test)
test_data_accuracy = accuracy_score(X_Test_prediction, Y_Test)

In [53]:
print('Accuracy on testing data : ', test_data_accuracy)

Accuracy on testing data :  0.9942411057077041
